# Loading the data


In [1]:
%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics 
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# connection to the database
con = sqlite3.connect('amazon_data/database.sqlite')

# filtering psitive and negative reviews
filtered_data = pd.read_sql_query("""
SELECT * FROM Reviews WHERE Score != 3
""",con)

OperationalError: ignored

In [0]:
filtered_data

In [0]:
# seperate positive and negative reviews 
def partition(x):
    if x<3:
        return 'negative'
    return 'positive'


In [0]:
filtered_data.shape

In [0]:
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative
# filtered_data['Score']

In [0]:
filtered_data.head()

# Data cleaning: deduplication

In [0]:
display = pd.read_sql_query("""
SELECT * FROM Reviews WHERE Score != 3 AND UserId ="AR5J8UI46CURR"
ORDER BY ProductID
""",con)
display

In [0]:
# first sorting data according to productid in ascdending order
sorted_data = filtered_data.sort_values('ProductId',axis=0,ascending=True)


In [0]:
# remove duplicate entries 
final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first',inplace=False)
final.shape

In [0]:
# Checking what % of data still remains 
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

In [0]:
# some data have helpfulnessdenominator less than helpfulnessnumerator
display = pd.read_sql_query("""
SELECT * FROM Reviews WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""",con)
display

In [0]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]

In [0]:
print(final.shape)

In [0]:
final['Score'].value_counts()

# Bag of words

In [0]:
count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(final['Text'].values)

In [0]:
type(final_counts)

In [0]:
final_counts.get_shape()

# Text preprocessing (Stemming, stopword removal and lemmatization)

In [0]:
# find sentences containing HTML pages
import re
i =0 
for sent in final['Text'].values:
    if(len(re.findall('<.*?>',sent))):
        print(i)
        print(sent)
        break;
    i+=1;

In [0]:
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('stopwords')

In [0]:
stop = set(stopwords.words('english'))#set of stopwords
stop

In [0]:
# initializing a snowball stemmer
sno = nltk.stem.SnowballStemmer('english')
sno

In [0]:
# function to clean the word of any html tags
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,' ',sentence)
    return cleantext

In [0]:
# clean word of any puctuations
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)    
    cleaned = re.sub(r'[.|,|)|(|\|/]',r'',cleaned)
    return cleaned

In [0]:
print(sno.stem('tasty'))

In [0]:
# code for implementing step by step checks
i=0
str1=' '
final_string=[]
all_positive_words=[]
all_negative_words=[]
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    for w in cleanhtml(sent).split():
        for cleaned_words in cleanpunc(w).split():
            print(cleaned_words)
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
                s = (sno.stem(cleaned_words.lower())).encode('utf8')
                filtered_sentence.append(s)
                if(final['Score'].values)[i] == 'positive':
                    all_positive_words.append(s)
                if(final['Score'].values)[i] == 'negative':
                    all_negative_words.append(s)
            else:
                continue
        else:
            continue
    str1= b" ".join(filtered_sentence)
    final_string.append(str1)
    i+=1
        
        
        
        
        

In [0]:
final['CleanedText'] = final_string #adding column of cleaned text

In [0]:
final.head(3)

In [0]:
# store the table into a sqlite database table for future
conn = sqlite3.connect('final.sqlite')
c= conn.cursor()
conn.text_factory = str
final.to_sql('Reviews',conn,flavor-None,schema=None,if_exists='replace')

# bigrams ngrams 

In [0]:
freq_dist_positive=nltk.FreqDist(all_positive_words)
freq_dist_negative=nltk.FreqDist(all_negative_words)
print("Most common frequent words:",freq_dist_positive.most_common(20))
print("Most common negative words:",freq_dist_negative.most_common(20))

In [0]:
# removing stop words like not should be avoided before building ngrams
count_vect = CountVectorizer(ngram_range(1,2))
final_bigram_counts = count_vect.fit_transform(final['Text'].values)

In [0]:
final_bigram_counts.get_shape() 

# tf-idf

In [0]:
tf_idf_vect = TfidfVectorizer(n_gram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [0]:
final_tf_idf.shape()

In [0]:
features = tf_idf_vect.get_feature_names()
len(features)

In [0]:
features[100000:100010]

In [0]:
print(final_tf_idf[3,:].toarray()[0])

In [0]:
def top_tf_idf_feats(row,features,top_n=25):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i],row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature','tfidf']
    return df

top_tfidf = top_tf_idf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [0]:
top_tfidf

# word2vec


In [0]:
# using google word2vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

# model = KeyedVectors.load_word2vec_format('....')

In [0]:
# model.wv['computer']
# model.wv.similarity('woman','man')
# model.wv.most_similar('woman')

In [0]:
# train your own word2vec model
import gensim
i=0
list_of_sent=[]

for sent in final['Text'].values:
    filtered_sentence=[]
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sent.append(filtered_sentence)

In [0]:
print(final['Text'].values[0])
print("***************************************************************")
print(list_of_sent[0])

In [0]:
w2v_model = gensim.models.Word2Vec(list_of_sent,min_count=5,size=50,workers=4)

In [0]:
words = list(w2v_model.wv.vocab)
print(len(words))

In [0]:
w2v_model.wv.most_similar('tasty')

In [0]:
count_vect_feat = count_vect.get_feature_names()
count_vect_feat.index('like')
print(count_vect_feat[64055])

# avg w2v, tfidf-w2v

In [0]:
# avg word2vec
sent_vectors = []
for sent in list_of_sent:
    sent_vec=np.zeros(50)
    cnt_words=0
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words+=1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
    
print(len(sent_vectors))
print(len(sent_vectors[0]))


In [0]:
# tf-idf weighted word2vec
tfidf_feat = tf_idf_vect.get_features_names()

tfidf_sent_vectors = []
row=0
for sent in list_of_sent:
    sent_vec=np.zeros(50)
    weighted_sum=0
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            tfidf = final_tf_idf[row,tfidf_feat.index(word)]
            sent_vec += (vec*tf_idf)
            weighted_sum += tf_idf
        except:
            pass
    sent_vec /= weighted_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1
